# Part I. ETL Pipeline for Pre-Processing the Files

## Pre-processiong of the files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from decimal import Decimal

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

# DEBUG: Print the result
# print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:
    # Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # Creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        # Extracting each data row one by one and append it to the common array     
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# DEBUG: get total number of rows 
#print(len(full_data_rows_list))

# DEBUG: get the list of event data rows 
# print(full_data_rows_list)

# DEBUG: get the first record
#print(full_data_rows_list[0])

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the 
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# DEBUG: get the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## The file <font color=red>event_datafile_new.csv</font>, located within the Workspace directory, contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Configuration of the cluster:

#### Creating a Cluster

In [5]:
# Connection to a Cassandra instance in your local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# Establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_events 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above ('music_events')
try:
    session.set_keyspace('music_events')
except Exception as e:
    print(e)

### ***Query 1:*** Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
## Query 1:  Get the following: the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

createQuery1 = """CREATE TABLE IF NOT EXISTS data_by_sessionId_itemInSession (sessionId int, itemInSession int, artist text, song_title text, song_length decimal, PRIMARY KEY (sessionId, itemInSession))"""  
session.execute(createQuery1)

In [9]:
# Read lines from CSV file and create the INSERT query
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        insertQuery1 = "INSERT INTO data_by_sessionId_itemInSession (sessionId, itemInSession, artist, song_title, song_length)"
        insertQuery1 = insertQuery1 + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(insertQuery1, (int(line[8]), int(line[3]), line[0], line[9], Decimal(line[5])))

#### Check the procedure with a SELECT query, whose result must be:
<img src="images/result_query1.PNG" width="900" height="350">

In [10]:
## SELECT statement to verify the data was entered into the table 'data_by_sessionId_itemInSession'

selectQuery1 = """SELECT artist, song_title,song_length FROM data_by_sessionId_itemInSession WHERE sessionId=338 AND itemInSession=4"""
try:
    rows = session.execute(selectQuery1)
except Exception as e:
    print(e)

print("** The result of the query must include only 1 record: **")
for row in rows:
    print (row)

** The result of the query must include only 1 record: **
Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=Decimal('495.3073'))


### ***Query 2:*** Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
## Query 2: Get the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

createQuery2 = """CREATE TABLE IF NOT EXISTS data_by_userId_sessionId_itemInSession \
    (userId int, sessionId int, itemInSession int, artist text, first_name text, last_name text, song text, PRIMARY KEY ((userId, sessionId), itemInSession))"""  
session.execute(createQuery2)                

In [14]:
# Read lines from CSV file and create the INSERT query
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        insertQuery2 = "INSERT INTO data_by_userId_sessionId_itemInSession (userId, sessionId, itemInSession, artist, first_name, last_name, song)"
        insertQuery2 = insertQuery2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(insertQuery2, (int(line[10]),int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))

#### Check the procedure with a SELECT query, whose result must be:
<img src="images/result_query2.PNG" width="900" height="350">

In [15]:
## SELECT statement to verify the data was entered into the table 'data_by_userId_sessionId_itemInSession'

selectQuery2 = """SELECT artist, first_name, last_name, song FROM data_by_userId_sessionId_itemInSession WHERE userId=10 AND sessionId=182"""
try:
    rows = session.execute(selectQuery2)
except Exception as e:
    print(e)

print("** The result of the query must include only 4 record: **")
for row in rows:
    print (row)

** The result of the query must include only 4 record: **
Row(artist='Down To The Bone', first_name='Sylvie', last_name='Cruz', song="Keep On Keepin' On")
Row(artist='Three Drives', first_name='Sylvie', last_name='Cruz', song='Greece 2000')
Row(artist='Sebastien Tellier', first_name='Sylvie', last_name='Cruz', song='Kilometer')
Row(artist='Lonnie Gordon', first_name='Sylvie', last_name='Cruz', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


### ***Query 3:*** Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

createQuery3 = """CREATE TABLE IF NOT EXISTS data_by_song \
    (song text, userId int, first_name text, last_name text, PRIMARY KEY (song, userId))"""  
session.execute(createQuery3)         

In [17]:
# Read lines from CSV file and create the INSERT query
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        insertQuery3 = "INSERT INTO data_by_song (song, userId, first_name, last_name)"
        insertQuery3 = insertQuery3 + " VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(insertQuery3, (line[9], int(line[11]), line[1], line[4]))

IndexError: list index out of range

#### Check the procedure with a SELECT query, whose result must be:
<img src="images/result_query3.PNG" width="900" height="350">

In [ ]:
## SELECT statement to verify the data was entered into the table 'data_by_song'

selectQuery3 = """SELECT first_name, last_name FROM data_by_song WHERE song='All Hands Against His Own'"""
try:
    rows = session.execute(selectQuery3)
except Exception as e:
    print(e)

print("** The result of the query must include only 3 record: **")
for row in rows:
    print (row)

## Destruction of all the configuration created in the process:

### Drop all the tables created before closing out the sessions

In [ ]:
dropQuery1 = "drop table data_by_sessionId_itemInSession"
try:
    rows = session.execute(dropQuery1)
except Exception as e:
    print(e)

dropQuery2 = "drop table data_by_userId_sessionId_itemInSession"
try:
    rows = session.execute(dropQuery2)
except Exception as e:
    print(e)

dropQuery3 = "drop table data_by_song"
try:
    rows = session.execute(dropQuery3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()